<a href="https://colab.research.google.com/github/ccasanoval/RLtests/blob/master/DoomV4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DoomV4

RL  = Stable Baseline 3 : PPO

ENV = Gymnasium + VizDoom

URL = https://github.com/AKapich/Reinforcement_Learning_Doom

URL = https://stable-baselines3.readthedocs.io/en/master/

In [10]:
!pip install vizdoom
!pip install stable_baselines3

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
############### GYM ENV == VIZ DOOM ###########################################
# config_file_path = os.path.join(vzd.scenarios_path, "defend_the_center.cfg")
# game = vzd.DoomGame()
# game.load_config(config_file_path)
# game.set_window_visible(False)
# game.set_mode(vzd.Mode.PLAYER)
# game.set_screen_format(vzd.ScreenFormat.GRAY8)
# game.set_screen_resolution(vzd.ScreenResolution.RES_640X480)#cnfg=screen_resolution = RES_320X240
# #game.add_game_args("doom_skill=1")
# game.init()
from vizdoom import DoomGame
import numpy as np
from gymnasium import Env
from gymnasium.spaces import Discrete, Box
import cv2

class VizDoomGym(Env):
    def __init__(self, scenario, render=True, number_of_actions=3):

        self.game = DoomGame()
        self.game.load_config(f"{scenario}.cfg")

        self.game.set_window_visible(render)
        self.game.init()

        self.observation_space = Box(
            low=0, high=255, shape=(100, 160, 1), dtype=np.uint8
        )
        self.number_of_actions = number_of_actions
        self.action_space = Discrete(number_of_actions)

    def step(self, action):
        actions = np.identity(self.number_of_actions)
        reward = self.game.make_action(actions[action], 4)

        if self.game.get_state():
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            info = self.game.get_state().game_variables[0]  # ammo
        else:
            state = np.zeros(self.observation_space.shape)
            info = 0

        info = {"info": info}
        terminated = self.game.is_episode_finished()

        truncated = (
            self.game.is_player_dead()
            or self.game.is_player_dead()
            or self.game.is_player_dead()
        )

        return state, reward, terminated, truncated, info

    def reset(self, seed=0):
        self.game.new_episode()
        state = self.game.get_state().screen_buffer

        if self.game.get_state():
            info = self.game.get_state().game_variables[0]  # ammo
        else:
            info = 0

        return (self.grayscale(state), {"ammo": info})

    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160, 100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100, 160, 1))
        return state

    def close(self):
        self.game.close()


In [12]:
##################### SB3 : CALLBACK ##########################################
from stable_baselines3.common.callbacks import BaseCallback
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, verbose=1), rltype="?":
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = 'best_model_{}_{}'.format(self.rltype, self.n_calls)
            self.model.save(model_path)

        return True


In [13]:
###################### TRAIN == SB3 #######################################
from stable_baselines3 import PPO
from stable_baselines3 import DQN

from enum import Enum
class ModelType(Enum):
    PPO = 1
    DQN = 2

modelType = ModelType.PPO
modelNew = True
modelTrain = True

#####################

callback = TrainAndLoggingCallback(check_freq=10000, rltype=modelType.name)
env = VizDoomGym(render=False, scenario='defend_the_center')

match modelType:
    case ModelType.PPO:
        if not modelNew:
            model = PPO.load('best_model_PPO', env=env)
        else:
            model = PPO(
                "CnnPolicy",
                env,
                verbose=1,
                seed=0,
                #tensorboard_log=".",
                #learning_rate=0.0001,
                #n_steps=4096
            )
    case ModelType.DQN:
        if not modelNew:
            model = DQN.load('best_model_DQN', env=env)
        else:
            model = DQN(
                "CnnPolicy",
                env,
                verbose=1,
                seed=0,
                #learning_rate=0.0001,
            )

# TRAIN
if modelTrain:
    model.learn(total_timesteps=150000, callback=callback)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 80.2     |
|    ep_rew_mean     | 0.04     |
| time/              |          |
|    fps             | 228      |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 81.2        |
|    ep_rew_mean          | 0.14        |
| time/                   |             |
|    fps                  | 145         |
|    iterations           | 2           |
|    time_elapsed         | 28          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009308739 |
|    clip_fraction        | 0.0394      |
|    clip_range      

In [14]:
###################### TEST == SB3 ########################################
from stable_baselines3.common.evaluation import evaluate_policy

env = VizDoomGym(render=True, scenario='defend_the_center')
match modelType:
    case ModelType.PPO:
        model = PPO.load('best_model_PPO', env=env)
    case ModelType.DQN:
        model = DQN.load('best_model_DQN', env=env)

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
env.close()

print(f"mean_reward:{mean_reward:.2f}")
print(f"std_reward:{std_reward:.2f}")


ViZDoomErrorException: Could not initialize SDL video:
No available video device
